In [ ]:
# Cacul de la pertinence de l'offre 1 et 6 : 1-2= -1 
# Peu importe le choix des offres choisies il faut ajouter une pertinence de -1 au calcul total de la pertinence

 ![](https://i.imgur.com/vZzyTTy.png)

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Créationd ddu répertoire pertinence (directory)
print(os.listdir("../input/pertinence"))

In [ ]:
#Importation de la table que nous avons crée sous format csv 
import pandas as pd 
data=pd.read_csv("../input/pertinence/pertinence.csv")
data

In [ ]:
# Transformation de la colonne Offre en index.
data.set_index("Offre")

In [ ]:
# Somme de la colonne Pertinence si le prospect accède à l'ensemble des 6 offres 
data["Pertinence"].sum()

In [ ]:
# Sur les 6 offres le prospect en choisit obligatoirement au minimum 2 : la première offre et la dernière offre (ici respectivement 1 et 6)
# Le prospect aura au minimum 2 offres et au maximum 6 offres: [2;6]
# On va donc raisonner en termes de nombres d'offres prises, s'il choisit une offre supplementaire : cela fera 2+1=3 donc 3 offres et il a sauté 3 
# offres
# hormis l'offre n°1 et l'offre n°6, il peut donc choisir des offres supplémentaires du lot [2,3,4,5] 

longueur=len(data)+1
List1=list(range(longueur))
List1=List1[2:-1]
List1

In [ ]:
# Voici les deux offres obligatoires
ee=list(range(longueur))
eee=[ee[1],ee[-1]]
eee


In [ ]:
# Pertinence des offres obligatoire
dern=data[data["Offre"].isin(eee)][["Pertinence"]].sum()
dern

In [ ]:
import itertools

stuff = [1, 2, 3]
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        print(subset)

In [ ]:
import itertools

stuff = [1, 2, 3]
for L in range(0, 4):
    for subset in itertools.combinations(stuff, L):
        print(subset)


In [ ]:
stuff = [1, 2, 3]
len(stuff)

In [ ]:
from itertools import combinations 
  
# Get all combinations of [2, 1, 3] 
# and length 2 

comb = combinations(List1, 1) 
  
# Print the obtained combinations 
for i in list(comb): 
    A=i
    B=data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern
    
    print(A,B)





In [ ]:
from itertools import combinations 
comb = combinations(List1, 1) 
lst=[max(data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern) for i in list(comb)]




In [ ]:
lst

In [ ]:
from itertools import combinations 
comb = combinations(List1, 1) 
lst1=[i for i in list(comb)]




In [ ]:
import pandas as pd
 
df = pd.DataFrame({'key1': [], 'key2': []})

df = df.append({'key1': lst, 'key2': lst1}, ignore_index=True)

In [ ]:
df

In [ ]:

df=pd.DataFrame(lst,columns=['col1'])
df1=pd.DataFrame(lst1,columns=['col2'])


In [ ]:
DF= pd.DataFrame(
    {'lst1Title': lst,
     'lst2Title': lst1
    })
DF[DF["lst1Title"]==DF["lst1Title"].max()]

In [ ]:
DF[DF["lst1Title"]==DF["lst1Title"].max()]

In [ ]:
import pandas as pd
 
df = pd.DataFrame({'key1': [], 'key2': [], 'key3': []})
for i in range(10):
    df = df.append({'key1': i, 'key2': i*2, 'key3': i**3}, ignore_index=True)

In [ ]:
df

In [ ]:
df

In [ ]:
from itertools import combinations 
  
# Get all combinations of [2, 1, 3] 
# and length 2 

comb = combinations(List1, 1) 
  
# Print the obtained combinations 
for i in list(comb): 
    A=i
    B=data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern
    
    print(A)


    
import pandas as pd
 
df = pd.DataFrame({'key1': []})
for i in list(comb):
    df = df.append({'key1': i}, ignore_index=True)

In [ ]:
df

In [ ]:
for y in [0,8] : 
   for z in [0,1,2,3] :
     x=y+z

     print x


lst = [y+z for y in [0,8] for z in [0,1,2,3]]
print(lst) # prints:  [1,2,3,8,9,10,11]

In [ ]:
lst

In [ ]:
List1

In [ ]:
from itertools import combinations 
  


comb = combinations(List1, 1) 
  
# Print the obtained combinations 
for i in list(comb): 
    A=i
    B=data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern

    print(A,B)
   

In [ ]:
import itertools 

for L in range(0,len(List1)+1):
    for subset in itertools.combinations(List1,L):
        print(subset)
        A=data[data["Offre"].isin(subset)][["Pertinence"]].sum()+dern
        print(A)
        

In [ ]:
from itertools import combinations 
  
# Get all combinations of [2, 1, 3] 
# and length 2 

comb = combinations(List1, 2) 
  
# Print the obtained combinations 
for i in list(comb): 
    print (i)
    print(data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern)

In [ ]:
from itertools import combinations 
  
# Get all combinations of [2, 1, 3] 
# and length 2 

comb = combinations(List1, 3) 
  
# Print the obtained combinations 
for i in list(comb): 
    print (i) 
    print(data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern)

In [ ]:
from itertools import combinations 
  
# Get all combinations of [2, 1, 3] 
# and length 2 

comb = combinations(List1, 4) 
  
# Print the obtained combinations 
for i in list(comb): 
    print (i) 
    print(data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern)

In [ ]:
from itertools import combinations 
  
# Get all combinations of [2, 1, 3] 
# and length 2 

comb = combinations(List1, 1) 
  
# Print the obtained combinations 
for i in list(comb): 
    A=i
    B=data[data["Offre"].isin(i)][["Pertinence"]].sum()+dern

    print(A,B)

In [ ]:
import itertools 

for L in range(0,len(List1)+1):
    for subset in itertools.combinations(List1,L):
        print(subset)
        A=data[data["Offre"].isin(subset)][["Pertinence"]].sum()+dern
        print(A)
        

In [ ]:


lst=[max(data[data["Offre"].isin(subset)][["Pertinence"]].sum()+dern) for L in range(0,len(List1)+1) for subset in itertools.combinations(List1,L)]








In [ ]:
lst

In [ ]:

lst1=[subset for L in range(0,len(List1)+1) for subset in itertools.combinations(List1,L)]


In [ ]:
lst1

In [ ]:
import itertools 

for L in range(0,len(List1)+1):
    for subset in itertools.combinations(List1,L):
        print(subset)
        

In [ ]:
DF= pd.DataFrame(
    {'col1': lst,
     'col2': lst1
    })
DF


In [ ]:
DF['col2'] = '(1,6)+' + DF['col2'].astype(str)

In [ ]:
DF

In [ ]:
DF[DF["col1"]==DF["col1"].max()]

In [ ]:
# Donc la meilleur pertinence est atteinte lorsque le prospect choisit les offres : 
- 1 ,6,4 
- 1, 6, 2 et 4 